In [1]:
import pandas as pd
import numpy as np
import datetime
from pyecharts.charts import *
from pyecharts.globals import ThemeType
from pyecharts.charts import Bar
from pyecharts.charts import Funnel
from pyecharts import options as opts

# 项目背景
商品子集都是偏服务类的商品，涵盖阿里巴巴集团十个主要的商品大类，例如汽车售后服务、摄影服务、餐饮、电影等，其特色是线上购买、线下服务。

# 分析框架

## 流量（入口）
1-1、pv、uv的趋势对比【折线图】-1212活动前后的每日流量对比
流量-影响因素
1-2、新增pv、uv的趋势对比【折线图】 -近30天主图吸引力、广告营销（渠道引流）、市场竞争等主要因素对pv、uv的每日影响
流量-影响销量
1-3、活动前后购买率（buy/pv）、购买数量的对比【折线、柱形图】 -1212活动前后销量对比

## 用户行为偏好（时间维度）
2、日时段各行为数量的走势和对比 【折线图】 -利用活跃时点安排好商品上新、页面更换、调配客服

## 运营环节（转化率分析）
3-1、活动前后用户转化率对比【漏斗图】 -优化悬崖式下跌的行为对应的商品运营环节

双12期间流量/订单量top10类目

In [2]:
df = pd.read_csv('./commodity_user_behavior_category.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15463110 entries, 0 to 15463109
Data columns (total 6 columns):
user_id          int64
item_id          int64
behavior_type    int64
user_geohash     object
item_category    int64
time             object
dtypes: int64(4), object(2)
memory usage: 707.8+ MB


In [3]:
df.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,10001082,285259775,1,97lk14c,4076,2014-12-08 18
1,10001082,4368907,1,NaN,5503,2014-12-12 12
2,10001082,53616768,4,NaN,9762,2014-12-02 15
3,10001082,9947871,1,NaN,2825,2014-11-28 20
4,10001082,150720867,1,95qoghe,3200,2014-12-15 08


In [4]:
df.describe()

,user_id,item_id,behavior_type,item_category
count,1.546311e+07,1.546311e+07,1.546311e+07,1.546311e+07
mean,7.015207e+07,2.023169e+08,1.153780e+00,6.827181e+03
std,4.572019e+07,1.167524e+08,5.440445e-01,3.810410e+03
min,4.920000e+02,3.700000e+01,1.000000e+00,2.000000e+00
25%,3.021406e+07,1.014015e+08,1.000000e+00,3.687000e+03
50%,5.638708e+07,2.022669e+08,1.000000e+00,6.054000e+03
75%,1.166482e+08,3.035247e+08,1.000000e+00,1.025800e+04
max,1.424430e+08,4.045625e+08,4.000000e+00,1.408000e+04


# 数据清洗
## 删除重复值

In [5]:
df.drop_duplicates(inplace=True)
df.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time
0,10001082,285259775,1,97lk14c,4076,2014-12-08 18
1,10001082,4368907,1,NaN,5503,2014-12-12 12
2,10001082,53616768,4,NaN,9762,2014-12-02 15
3,10001082,9947871,1,NaN,2825,2014-11-28 20
4,10001082,150720867,1,95qoghe,3200,2014-12-15 08


In [6]:
df.isnull().sum()

user_id                0
item_id                0
behavior_type          0
user_geohash     8207386
item_category          0
time                   0
dtype: int64

将df.time转成日期格式

In [7]:
df['time']=pd.to_datetime(df['time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15463110 entries, 0 to 15463109
Data columns (total 6 columns):
user_id          int64
item_id          int64
behavior_type    int64
user_geohash     object
item_category    int64
time             datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 825.8+ MB


In [8]:
df['date'] = df.time.dt.date
df['hour'] = df.time.dt.hour
df.head()

,user_id,item_id,behavior_type,user_geohash,item_category,time,date,hour
0,10001082,285259775,1,97lk14c,4076,2014-12-08 18:00:00,2014-12-08,18
1,10001082,4368907,1,NaN,5503,2014-12-12 12:00:00,2014-12-12,12
2,10001082,53616768,4,NaN,9762,2014-12-02 15:00:00,2014-12-02,15
3,10001082,9947871,1,NaN,2825,2014-11-28 20:00:00,2014-11-28,20
4,10001082,150720867,1,95qoghe,3200,2014-12-15 08:00:00,2014-12-15,8


In [9]:
# 将某些字段转换成字符串格式
df['user_id'] = df.user_id.values.astype('str')
df['item_id'] = df.item_id.values.astype('str')
df['behavior_type'] = df.behavior_type.values.astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15463110 entries, 0 to 15463109
Data columns (total 8 columns):
user_id          object
item_id          object
behavior_type    object
user_geohash     object
item_category    int64
time             datetime64[ns]
date             object
hour             int64
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 1.0+ GB


In [10]:
df['item_category'] = df.item_category.values.astype('str')

# 数据可视化
## 每日PU、UV

In [11]:
#提取数据
pv_day = df[df.behavior_type =='1'].groupby('date')['behavior_type'].count()
uv_day = df[df.behavior_type =='1'].drop_duplicates(['user_id','date']).groupby('date')['user_id'].count()
#转换成图表所需的格式（list）
#1、日期（list.index）
date = pv_day.index
#2、pv、uv（list.values）
pv = np.around(pv_day.values/10000,decimals=2)
uv = np.around(uv_day.values/10000,decimals=2)

In [12]:
# 制作图表
x=list(date)
y1=pv
y2=uv
pvuv_day_line = (Line(init_opts=opts.InitOpts(theme=ThemeType.DARK)) #主题设置
       .add_xaxis(x) #x轴数据源
       .add_yaxis('pv',#图例名字
                  y1, #y1轴数据源
                  label_opts=opts.LabelOpts(is_show=False) #不显示数据标签
                 )
       .add_yaxis('uv',#图例名字
                  yaxis_index=1, #Y的双轴1号索引（区别于y1轴）
                  y_axis=y2, #y2轴数据源
                  label_opts=opts.LabelOpts(is_show=False) #不显示数据标签
                 ) 
        .extend_axis( #y2的轴设置 
                    yaxis=opts.AxisOpts(
                                        name='uv',#轴名字
                                        min_=0,#轴起点值
                                        max_=1.6, #轴最大值
                                        interval=0.4, #轴区间间隔
                                        axislabel_opts=opts.LabelOpts(formatter="{value} 万人") #轴数据标签格式设置
                                         )
                    )
        .set_global_opts( #全局设置
                        tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type='cross'), #随鼠标位置显示xy轴的数据、聚焦形式(交叉)
                        xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow")),#随鼠标位置凸显x轴长条、凸显形式（阴影）
                        yaxis_opts=opts.AxisOpts(name='pv',axislabel_opts=opts.LabelOpts(formatter="{value} 万次")),#y1轴(默认轴)名字、轴数据标签格式设置
                        title_opts=opts.TitleOpts(title="每日pv和uv") #标题        
                         )              
            )
pvuv_day_line.render_notebook() #展示图表

分析：

1、双12活动的流量主要集中于12月11日和12日，10日之前并没有太大的上涨，特别是uv的涨幅甚小。建议优化商品预热的活动方案，提前吸引人流进店挑选商品；

2、活动期间pv比uv波动较大，活动引流效应明显；

3、13日之后pv、uv均回落至比活动前稍高的水平；

## 每日pv、uv增量

### 双y轴双曲线图

In [13]:
# 提取数据
#联结pv、uv
pv_uv= pd.merge(pv_day,uv_day,on='date',how='outer')
#向下作差（后一日减前一日）
new_pv_uv = pv_uv.diff() 
new_pv_uv.columns=['new_pv','new_uv']
# 制作图表
x=new_pv_uv.index
y1=new_pv_uv.new_pv
y2=new_pv_uv.new_uv

new_pvuv_day_line = (Line(init_opts=opts.InitOpts(theme=ThemeType.DARK))
                       .add_xaxis(x) 
                       .add_yaxis('新增pv',
                                  y1, 
                                  label_opts=opts.LabelOpts(is_show=False) 
                                     )
                       .add_yaxis('新增uv',
                                  yaxis_index=1, 
                                  y_axis=y2, 
                                  label_opts=opts.LabelOpts(is_show=False) 
                                  ) 
                        .extend_axis( 
                                  yaxis=opts.AxisOpts(
                                                    name='新增uv(人)',
                                                    min_=-2000,
                                                    max_=1542, 
                                                #   interval=1000, 
                                                    axislabel_opts=opts.LabelOpts(formatter="{value}") 
                                                       )
                                        )
                        .set_global_opts( 
                                    tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type='cross'), 
                                    xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow")),
                                    yaxis_opts=opts.AxisOpts(name='新增pv(人次)',
                                                             min_=-350000,
                                                             max_=250000, 
                                                             interval=100000, axislabel_opts=opts.LabelOpts
                                                             (formatter="{value}")),
                                    title_opts=opts.TitleOpts(title="新增pv和uv")
                                        )              
                            )
new_pvuv_day_line.render_notebook()

分析：
pv、uv日变动轨迹大致趋同；12日，pv增量22078，uv增量1542，pv是uv的143倍

## 用户行为趋势对比

### 单轴曲线图

In [14]:
#制作每日用户行为总量的数据表
view_day = df[df.behavior_type =='1'].groupby('date')['behavior_type'].count()
col_day = df[df.behavior_type =='2'].groupby('date')['behavior_type'].count()
add_day = df[df.behavior_type =='3'].groupby('date')['behavior_type'].count()
buy_day = df[df.behavior_type =='4'].groupby('date')['behavior_type'].count()
behavier_day = pd.merge(view_day,col_day,how='outer',on='date').merge(add_day,how='outer',on='date').merge(buy_day,how='outer',on='date')
behavier_day.columns=['view','col','add','buy']
behavier_day.head()

,view,col,add,buy
date,,,,
2014-11-18,395982,13205,18703,6115
2014-11-19,405103,13432,18137,5859
2014-11-20,392599,13111,17883,5787
2014-11-21,372570,12606,17310,5701
2014-11-22,390638,12975,17323,5762


In [15]:
#制作图表
x=view_day.index.tolist() 
y2=col_day.values.tolist()
y3=add_day.values.tolist()
y4=buy_day.values.tolist()

behavier_day_line = (Line(init_opts=opts.InitOpts(theme=ThemeType.DARK))
       .add_xaxis(x)

       .add_yaxis('收藏',
                  y_axis=y2, 
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
       .add_yaxis('加购',
                  y_axis=y3, 
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
       .add_yaxis('购买',
                  y_axis=y4, 
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
        .set_global_opts( 
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type='cross'), 
            xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow")),
            yaxis_opts=opts.AxisOpts(name='人次',
                                     axislabel_opts=opts.LabelOpts(formatter="{value}")),
            title_opts=opts.TitleOpts(title="用户行为趋势对比")  
            
                        )              
            )
behavier_day_line.render_notebook()

分析：

1、活动前有两波收藏、加购小高峰（11.23-25,11.30-12.3）,邻近活动，加购数明显上涨，12当天购买人数超活动前的日均加购数。

## 日常和双12日均各时段用户行为趋势对比

### 曲线图*2

In [16]:
df['date'] = pd.to_datetime(df['date'])
#提取数据
#日常各时段行为总量
daily_df = df[~df['date'].isin(['2014-12-11','2014-12-12'])]
view_hour = daily_df[daily_df.behavior_type =='1'].groupby('hour')['behavior_type'].count()
col_hour = daily_df[daily_df.behavior_type =='2'].groupby('hour')['behavior_type'].count()
add_hour = daily_df[daily_df.behavior_type =='3'].groupby('hour')['behavior_type'].count()
buy_hour = daily_df[daily_df.behavior_type =='4'].groupby('hour')['behavior_type'].count()
#双12各时段行为总量 （日期说明：因双十二开启时间为12月12号零点，活动期间用户行为主要发生时间在11号到12号，因此本次活动分析时筛选的时间为这两天）
active_df = df[df['date'].isin(['2014-12-11','2014-12-12'])]
view_active_ahour = active_df[active_df.behavior_type =='1'].groupby('hour')['behavior_type'].count()
col_active_ahour = active_df[active_df.behavior_type =='2'].groupby('hour')['behavior_type'].count()
add_active_ahour = active_df[active_df.behavior_type =='3'].groupby('hour')['behavior_type'].count()
buy_active_ahour = active_df[active_df.behavior_type =='4'].groupby('hour')['behavior_type'].count()

In [17]:
# 制作图表
x=col_hour.index.tolist()
#日常
y2=np.around(col_hour.values/29,decimals=0).tolist()
y3=np.around(add_hour.values/29,decimals=0).tolist()
y4=np.around(buy_hour.values/29,decimals=0).tolist()
#双12
y5=np.around(col_active_ahour.values/2,decimals=0).tolist()
y6=np.around(add_active_ahour.values/2,decimals=0).tolist()
y7=np.around(buy_active_ahour.values/2,decimals=0).tolist()

daily_line = (Line()
       .add_xaxis(x) 
       .add_yaxis('收藏',
                  y_axis=y2,
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
       .add_yaxis('加购',
                  y_axis=y3,
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
       .add_yaxis('购买',
                  y_axis=y4,
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
        .set_global_opts( 
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type='cross'),
            legend_opts=opts.LegendOpts(pos_top='50%'), 
            xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow")),
            yaxis_opts=opts.AxisOpts(name='人次',
                                     axislabel_opts=opts.LabelOpts(formatter="{value}")),
            title_opts=opts.TitleOpts(title="日常日均各时段用户行为",pos_top='45%')   
                        )              
            )

active_line = (Line()
       .add_xaxis(x) 
       .add_yaxis('收藏',
                  y_axis=y5, 
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
       .add_yaxis('加购',
                  y_axis=y6, 
                  label_opts=opts.LabelOpts(is_show=False)
                 ) 
       .add_yaxis('购买',
                  y_axis=y7, 
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
        .set_global_opts( 
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type='cross'), 
            legend_opts=opts.LegendOpts(pos_top='5%'),
            xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow")),
            yaxis_opts=opts.AxisOpts(name='人次',
                                     axislabel_opts=opts.LabelOpts(formatter="{value}")),
            title_opts=opts.TitleOpts(title="双12日均各时段用户行为",pos_top='0%')        
                        )              
            )

ggrid = ( #拼图
    Grid(init_opts=opts.InitOpts(theme=ThemeType.DARK)) #统一设置主题
    .add(active_line, grid_opts=opts.GridOpts(pos_bottom="60%")) #位置
    .add(daily_line, grid_opts=opts.GridOpts(pos_top="60%")) #位置
)
ggrid.render_notebook()

分析：

1、无论是日常还是活动，用户行为高峰发生在每日的19时至第二天0时。建议：运营部尽量在19时之前进行商品上新或活动发布；

2、双12加购和购买最高峰均在0时，日常加购和购买最高峰均在21时。建议在加购、购买最高峰时段前1-2小时加大直播力度促成交易。

## 日常和双12日均各时段pv趋势对比

### 条形图

In [19]:
# 制作图表
x=col_hour.index.tolist()
y1=np.around(view_hour.values/29,decimals=0).tolist()
y8=np.around(view_active_ahour.values/2,decimals=0).tolist()

bar=(
    Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
    .add_xaxis(xaxis_data=x)
    .add_yaxis(
    "日常PV",
        y1,
        stack='stack1',
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
    "双12PV",
        y8,
        stack='stack1',
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="日常和双12每日时段PV走势对比"),
        legend_opts=opts.LegendOpts(pos_top='5%'),
        yaxis_opts=opts.AxisOpts(name='人次',
                                axislabel_opts=opts.LabelOpts(formatter="{value}"))
    )
)

bar.render_notebook()

分析：

1、流量高峰主要在19点开始，23点后下降。建议尽量在19时之前更换好优化后的商品主图、详情页等页面，吸引更多的人流。

## 不同时段购买率

### 双轴折线图

In [20]:
#准备数据
#日常各时段购买率
daily_df = df[~df['date'].isin(['2014-12-11','2014-12-12'])]
view_user_num = daily_df[daily_df.behavior_type =='1'].drop_duplicates(['user_id','date','hour']).groupby('hour')['behavior_type'].count()
buy_user_num = daily_df[daily_df.behavior_type =='4'].drop_duplicates(['user_id','date','hour']).groupby('hour')['behavior_type'].count()
daily_buy_rate = buy_user_num/view_user_num
#双12各时段购买率
active_df = df[df['date'].isin(['2014-12-11','2014-12-12'])]
view_active_user_num = active_df[active_df.behavior_type =='1'].drop_duplicates(['user_id','date','hour']).groupby('hour')['behavior_type'].count()
buy_active_user_num = active_df[active_df.behavior_type =='4'].drop_duplicates(['user_id','date','hour']).groupby('hour')['behavior_type'].count()
acitve_buy_rate = buy_active_user_num/view_active_user_num

In [21]:
x=view_hour.index.tolist() 
y1 = np.around(daily_buy_rate,decimals=2).tolist()
y2 = np.around(acitve_buy_rate,decimals=2).tolist()

buy_rate_line = (Line(init_opts=opts.InitOpts(theme=ThemeType.DARK))
       .add_xaxis(x) 
       .add_yaxis('日常购买率',
                  y_axis=y1, 
                  label_opts=opts.LabelOpts(is_show=True) 
                 )
       .add_yaxis('双12购买率',
                  y_axis=y2, 
                  label_opts=opts.LabelOpts(is_show=True) 
                 ) 
        .set_global_opts( 
            tooltip_opts=opts.TooltipOpts(is_show=True,trigger="axis",axis_pointer_type='cross'),
            xaxis_opts=opts.AxisOpts(type_='category',axispointer_opts=opts.AxisPointerOpts(is_show=True,type_="shadow")),
            yaxis_opts=opts.AxisOpts(name=' ',axislabel_opts=opts.LabelOpts(formatter="{value}")),
            title_opts=opts.TitleOpts(title="不同时段购买率"), 
            legend_opts=opts.LegendOpts(pos_top='5%') 
                        )              
            )
buy_rate_line.render_notebook()

分析：

1、双12期间，购买率均超日常购买率；双12期间购买率高峰在0时，日常购买率高峰在10时以后，凌晨最低。

2、凌晨时段购买率均远超日常,结合上图（日常和双12日均各时段pv趋势对比）,凌晨PV流量是日常的两倍左右，说明活动期间凌晨时段有营销空间，建议商家在合规的情况下为凌晨购买的活跃消费者延长活动时间，促成更多的交易。



## 用户行为日常转化情况
### 漏斗图

In [22]:
#提取数据
#日常各时段行为数量
daily_df = df[~df['date'].isin(['2014-12-11','2014-12-12'])]
view_num = daily_df[daily_df.behavior_type =='1']['behavior_type'].count()
col_num = daily_df[daily_df.behavior_type =='2']['behavior_type'].count()
add_num = daily_df[daily_df.behavior_type =='3']['behavior_type'].count()
buy_num = daily_df[daily_df.behavior_type =='4']['behavior_type'].count()

#日常转化率
view_rate = view_num/view_num
col_rate = col_num/view_num
add_rate = add_num/view_num
buy_rate = buy_num/view_num

#设置漏斗的data_pair参数的list
behavier_labels=['浏览','加购','收藏','购买'] #行为标签
daily_behavier_rate = [np.around(view_rate*100,2), #转化率
                       np.around(add_rate*100,2),
                       np.around(col_rate*100,2),
                       np.around(buy_rate*100,2),]

daily_conversion_rate =(
            Funnel(init_opts=opts.InitOpts(theme=ThemeType.DARK))
            .add(
                series_name='用户行为',
                data_pair=[[behavier_labels[i],daily_behavier_rate[i]] for i in range(len(behavier_labels))], #list形式!!!
                gap=4, #漏斗间隔
                tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%",is_show=True), #工具提示的格式设置
                label_opts=opts.LabelOpts(is_show=True, position="ourside")
                )
            .set_global_opts(title_opts=opts.TitleOpts(title="日常用户转化率"))
)
daily_conversion_rate.render_notebook()

In [23]:
#提取数据
#双12活动各时段行为数量
active_df = df[df['date'].isin(['2014-12-11','2014-12-12'])]
view_active_num = active_df[active_df.behavior_type =='1']['behavior_type'].count()
col_active_num = active_df[active_df.behavior_type =='2']['behavior_type'].count()
add_active_num = active_df[active_df.behavior_type =='3']['behavior_type'].count()
buy_active_num  = active_df[active_df.behavior_type =='4']['behavior_type'].count()

# 活动转化率
view_active_rate = view_active_num/view_active_num
col_active_rate = col_active_num/view_active_num
add_active_rate = add_active_num/view_active_num
buy_active_rate = buy_active_num/view_active_num

#设置漏斗的data_pair参数的list
behavier_labels=['浏览','加购','收藏','购买']
active_behavier_rate =[np.around(view_active_rate*100,2),
                       np.around(add_active_rate*100,2),
                       np.around(col_active_rate*100,2),
                       np.around(buy_active_rate*100,2),]

active_conversion_rate =(
            Funnel(init_opts=opts.InitOpts(theme=ThemeType.DARK))
            .add(
                series_name='用户行为',
                data_pair=[[behavier_labels[i],active_behavier_rate[i]] for i in range(len(behavier_labels))], #list形式!!!
                gap=4, #漏斗间隔
                tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}%",is_show=True),
                label_opts=opts.LabelOpts(is_show=True, position="ourside")
                )
            .set_global_opts(title_opts=opts.TitleOpts(title="双12用户转化率"))
)

active_conversion_rate.render_notebook()

分析：

1、日常和双12活动的转化率大致相似，加购、收藏和购买转化率均很低，引流效果不佳，流失率高。建议：1.优化广告渠道，提高投放精准度，提高渠道引流来的用户质量；2.借鉴优秀同行产品，优化商品主图，提高产品视觉吸引力，增加转换率。

## 双12行业热度top10排名和分析

### 条形图*2

In [24]:
#提取数据
#1、流量：类目的pv top10
active_df = df[df['date'].isin(['2014-12-11','2014-12-12'])]
# 降序
view_active_num = active_df[active_df.behavior_type =='1'].groupby('item_category')['behavior_type'].count().sort_values(ascending=False)
# 取前十名
x1= view_active_num.index.tolist()[0:10]
y1=view_active_num.values.tolist()[0:10]

#2、购买增长率：双12类目下单量top10 及其日均下单量增长率 对比
# 双12购买数
active_df = df[df['date'].isin(['2014-12-11','2014-12-12'])]
buy_active_num =active_df[active_df.behavior_type =='4'].groupby('item_category')['behavior_type'].count().reset_index()
#日常购买数
daily_df = df[~df['date'].isin(['2014-12-11','2014-12-12'])]
buy_num = daily_df[daily_df.behavior_type =='4'].groupby('item_category')['behavior_type'].count().reset_index()
#合并
view_buy_avg = buy_active_num.merge(buy_num,how='inner',left_on='item_category',right_on='item_category')
view_buy_avg.columns=['item_category','buy_active_num','buy_num']
#计算增长率
view_buy_avg['buy_active_avg']=view_buy_avg['buy_active_num']/2
view_buy_avg['buy_avg']=view_buy_avg['buy_num']/29
view_buy_avg['growth_rate']=(view_buy_avg['buy_active_avg']-view_buy_avg['buy_avg'])/view_buy_avg['buy_avg']
# 降序
view_buy_avg.sort_values(by=['buy_active_num'],axis=0,ascending=False,inplace=True)
#取前十名
x2=view_buy_avg.item_category.tolist()[0:10]
y2=view_buy_avg.buy_active_num.tolist()[0:10]
y3=np.around(view_buy_avg.growth_rate*100,0).tolist()[0:10]

#制作图表
view_active_bar=(
    Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
    .add_xaxis(xaxis_data=x1)
    .add_yaxis(
    "pv",
        y1,
        color='rgb(300, 0, 100, 0.2)',
        label_opts=opts.LabelOpts(is_show=False)
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="双12商品类目pv TOP10"),
        legend_opts=opts.LegendOpts(pos_top='5%'),
        xaxis_opts=opts.AxisOpts(name=' ',name_location = "middle"),
        yaxis_opts=opts.AxisOpts(name='人次',
                                axislabel_opts=opts.LabelOpts(formatter="{value}"))
    )
)

buy_active_bar=(
    Bar(init_opts=opts.InitOpts(theme=ThemeType.PURPLE_PASSION))
    .add_xaxis(xaxis_data=x2)
    .add_yaxis(
    "下单量",
        y2,
        label_opts=opts.LabelOpts(is_show=False)
                )
     .add_yaxis('日均下单量增长率(与非活动日比较)',
                  yaxis_index=1, 
                  y_axis=y3, 
                  label_opts=opts.LabelOpts(is_show=False) 
                 ) 
      .extend_axis( 
            yaxis=opts.AxisOpts(
                name=' ',
                min_=100,
                max_=600,
                interval=100, 
                axislabel_opts=opts.LabelOpts(formatter="{value}%") 

                                )
                    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="双12商品类目下单量TOP10及日均下单量增长率",pos_top='48%'),
        legend_opts=opts.LegendOpts(pos_top='53%'),
        xaxis_opts=opts.AxisOpts(name='商品类目',name_location = "middle",name_gap=30),
        yaxis_opts=opts.AxisOpts(name=' ',is_show = True,
                                 min_=0,
                                 max_=800, 
                                 interval=200,                                
                                 axislabel_opts=opts.LabelOpts(formatter="{value}"))

                    )
     .set_series_opts(
                    markpoint_opts=opts.MarkPointOpts(data=[ #标记最值！
                        opts.MarkPointItem(type_="min",value_index=1,name="最小值"),
                        opts.MarkPointItem(type_="max",value_index=1,name="最大值")])
                     )
                )

ggrid = (
    Grid(init_opts=opts.InitOpts(theme=ThemeType.DARK))
    .add(view_active_bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
    .add(buy_active_bar, grid_opts=opts.GridOpts(pos_top="60%"))
        )

ggrid.render_notebook()

分析：

1、编号为11279、2825、10894的商品类目，虽然PV排前十，但下单量并没有排在前十，说明引流效果不佳，有待优化广告营销策略。

2、编号为6344、5027、13230、9516、1838的商品类目，均,属于下单量的top10，说明该商品类目销售无需太依赖pv流量，在广告方面的支出花费可能相对较少。

3、下单量排名第5到10名的商品类目，虽然销量不是最高，但相较于第3、4名，它们的双12下单量相较于日常的增长率均较高，说明这些类目的销售增长明显、热度较高，受到消费者的近期关注；建议该类目商家抓住机会加大广告营销、商品的研发，提高商家的知名度、口碑，提高服务质量。